In [2]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
from pyannote.audio import Pipeline
from pydub import AudioSegment
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
import os
import whisper
import warnings
import pandas as pd
import accelerate
import pprint 
pd.set_option('display.max_rows', 500)
warnings.filterwarnings('ignore')
 

In [3]:
#get video urls
speeches = pd.read_excel("speeches.xlsx")

In [3]:
#script to save audio files
def save_audio(url, output_path, filename):
    try:
        yt = YouTube(url)
        print(f"Downloading: {yt.title}")
        
        # Get audio-only stream
        ys = yt.streams.get_audio_only()
        
        # Ensure output directory exists
        os.makedirs(output_path, exist_ok=True)
        
        # Temporary file name (original format)
        temp_filename = f"{filename}.webm"
        temp_filepath = os.path.join(output_path, temp_filename)
        
        # Download audio in original format
        ys.download(output_path=output_path, filename=temp_filename)
        
        # Convert to MP3
        final_filename = f"{filename}.mp3"
        final_filepath = os.path.join(output_path, final_filename)
        audio = AudioSegment.from_file(temp_filepath)
        audio.export(final_filepath, format="mp3")
        
        # Remove the temporary file
        os.remove(temp_filepath)
        print(f"Downloaded and converted to MP3: {final_filename}")
    
    except Exception as e:
        print("----- Download failed: -----")
        print(f"URL: {url}")
        print(f"Error: {e}")


In [8]:
url_list = speeches["url"].to_list()

In [ ]:

for i, url in enumerate(url_list, start=1):
    output_path = "./audio_files"  
    filename = f"speech_{i}_{speeches["date"][i-1]}_{speeches["state"][i-1]}"
    save_audio(url, output_path=output_path, filename=filename)

Downloading: 🇺🇸 Donald Trump at Turning Point USA Rally in Phoenix, Arizona (June 6, 2024)
Downloaded and converted to MP3: speech_1_"2024-06-06"_AZ.mp3
Downloading: 🇺🇸 Donald Trump | Save America Rally at Sunset Park in Las Vegas, Nevada (June 9, 2024) [LIVE]
Downloaded and converted to MP3: speech_2_"2024-06-09"_NV.mp3
Downloading: 🇺🇸 Donald Trump | The People’s Convention by Turning Point USA in Detroit, Michigan (June 15, 2024)
Downloaded and converted to MP3: speech_3_"2024-06-15"_MI.mp3
Downloading: 🇺🇸 Donald Trump | Full Speech at Rally in Racine, Wisconsin (Subtitles) [Multilanguage CC]
Downloaded and converted to MP3: speech_4_"2024-06-18"_WI.mp3
Downloading: Donald Trump's speech Philadelphia June 22, 2024 I Temple University I @DonaldJTrumpforPresident
Downloaded and converted to MP3: speech_5_"2024-06-22"_PA.mp3
Downloading: Donald Trump campaign rally in Chesapeake, Virginia, one day after first 2024 presidential debate
Downloaded and converted to MP3: speech_6_"2024-06-28

In [4]:
#initiate model to script the audio files
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

print("Model and pipe has been initiated.")

Device set to use cpu


Model and pipe has been initiated.


In [11]:
for i in range(len(url_list)):
    if speeches["trimming_needed"][i] == "Yes":
        continue
    else:
        print(f"Generating transcript of speech_{i+1}_{speeches["date"][i]}_{speeches["state"][i]}.mp3")

        result_txt = pipe(f"./audio_files/speech_{i+1}_{speeches["date"][i]}_{speeches["state"][i]}.mp3", return_timestamps=True)
        text_to_add = list(result_txt.values())[0]

        print("Transcript has been generated.")
        os.makedirs("text_files", exist_ok=True)
        file_path = os.path.join("text_files", f"speech_{i+1}_{speeches["date"][i]}_{speeches["state"][i]}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(text_to_add)

        print("Transcript has been added to the folder.")


Generating transcript of speech_1_"2024-06-06"_AZ.mp3


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_2_"2024-06-09"_NV.mp3


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_3_"2024-06-15"_MI.mp3
Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_4_"2024-06-18"_WI.mp3


...

Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_66_"2024-11-02"_VA.mp3


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_67_"2024-11-02"_NC.mp3


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_68_"2024-11-03"_PA.mp3
Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_69_"2024-11-03"_NC.mp3
Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_70_"2024-11-03"_GA.mp3
Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_71_"2024-11-04"_NC.mp3
Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_72_"2024-11-04"_PA.mp3
Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_73_"2024-11-04"_PA.mp3
Transcript has been generated.
Transcript has been added to the folder.
Generating transcript of speech_74_"2024-11-04"_MI.mp3
Transcript has been generated.
Transcript has been added to the folder.
